# Produce decontextualized word embeddings

In this notebook, we will produce [decontextualized word embeddings][1] out of [the `roberta-base` model][2] and our fine-tuned version that can represent math-specific tokens.

 [1]: https://aclanthology.org/2021.wmt-1.112
 [2]: https://huggingface.co/roberta-base

In [1]:
! hostname

mir


In [ ]:
%%capture
! pip install .[transformers,scm]

In [ ]:
from gensim.models import KeyedVectors

## The `roberta-base` model

First, we will extract decontextualized word embeddings out of the `roberta-base` model.

In [ ]:
%%capture
! make decontextualized-word-embeddings-roberta-base

In [ ]:
%ls -lh decontextualized-word-embeddings-roberta-base

In [ ]:
KeyedVectors.load('decontextualized-word-embeddings-roberta-base').most_similar('integral')

## The tuned `roberta-base` model

Next, we will extract decontextualized word embeddings out of the `roberta-base` model fine-tuned so that it can represent math-specific tokens.

In [ ]:
%%capture
! make decontextualized-word-embeddings-tuned-roberta-base-text+latex

In [ ]:
%ls -lh decontextualized-word-embeddings-tuned-roberta-base-text+latex

In [ ]:
KeyedVectors.load('decontextualized-word-embeddings-tuned-roberta-base-text+latex').most_similar('integral')

In [ ]:
KeyedVectors.load('decontextualized-word-embeddings-tuned-roberta-base-text+latex').most_similar('F(x)')